Get required libraries

In [ ]:
pip install transformers datasets torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 81.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 41.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 90.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 71.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.4/269.4 kB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 17.7 MB/s eta 0:00:00


Load Google drive (where the data is stored)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Load data.
Train set is split into:
First 80% is train last 10% is evaluation

In [ ]:
from datasets import load_dataset
import os

current_wd = os.getcwd()
test_nl_dataset = load_dataset("json", data_files=f"{current_wd}/drive/MyDrive/thesis data/squad-nl/dev-v1.1.json", field="data", split='train')
test_en_dataset = load_dataset("json", data_files=f"{current_wd}/drive/MyDrive/thesis data/squad-en/dev-v1.1.json", field="data", split='train')
eval_dataset = load_dataset("json", data_files=f"{current_wd}/drive/MyDrive/thesis data/squad-nl/train-v1.1.json", field="data", split='train[:10%]')
train_dataset = load_dataset("json", data_files=f"{current_wd}/drive/MyDrive/thesis data/squad-nl/train-v1.1.json", field="data", split='train[-90%:]')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-3425e05f709b9f78/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e. Subsequent calls will reuse this data.


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-d7df6897fab42457/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e. Subsequent calls will reuse this data.


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-59cd0fa5bf152caa/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e. Subsequent calls will reuse this data.


Load in XLM Roberta as model (Autotokenizer in this case)

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

Setup Preprocess function that reads in the begin and end position of answers inside a paragraph (Source: [Huggingface](https://huggingface.co/learn/nlp-course/chapter7/7?fw=tf))

In [ ]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

Set datacollator

In [ ]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()


Preprocess the datasets

In [ ]:
tokenized_val = eval_dataset.map(preprocess_function, batched=True, remove_columns=eval_dataset.column_names)
tokenized_train = train_dataset.map(preprocess_function, batched=True, remove_columns=train_dataset.column_names)

Map:   0%|          | 0/8760 [00:00<?, ? examples/s]

Map:   0%|          | 0/78839 [00:00<?, ? examples/s]

Download the XLM-Roberta model

In [ ]:
import torch
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained("xlm-roberta-base")



Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForQuestionAnswering: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing XLMRobertaForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForQuestionAnswering were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream tas

Create a new trainer inherited class that uses GPU as standard

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

Set trainer and make sure it uses GPU

In [ ]:
training_args = TrainingArguments(
    output_dir=f"{current_wd}/drive/MyDrive/thesis data/models_nl",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=24,
    per_device_eval_batch_size=24,
    num_train_epochs=2,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.args.device

device(type='cuda', index=0)

Start training the model (Note, model was trained)

In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,1.357200,1.143780


Results:

1 	1.342200 	1.143171

2	x x

3	x x

Test model out on Dev set

In [ ]:
from transformers import pipeline
from datasets import load_metric
qa = pipeline("question-answering", model=f"{current_wd}/drive/MyDrive/thesis data/models/checkpoint-7000", device=0)
squad_metric = load_metric("squad")

preds = []
refs = []
for item in test_dataset:
  question = item["question"]
  context = item["context"]
  id = item["id"]
  pred_raw = qa(question=question, context=context)
  pred = {"prediction_text": pred_raw["answer"], "id": id}
  ref = {"answers": item["answers"], "id": id}
  refs.append(ref)
  preds.append(pred)

# print(test_dataset[0])
# print(preds[0])
squad_metric.compute(references=refs, predictions=preds)

  

<ipython-input-12-ea466c53bbf3>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  squad_metric = load_metric("squad")


/usr/local/lib/python3.9/dist-packages/transformers/pipelines/base.py:1070: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


{'exact_match': 70.51172707889125, 'f1': 81.53333285800063}

'exact_match': 70.51172707889125, 'f1': 81.53333285800063